In [ ]:
'''
Обработка текста предвыборных программ.

Цели:
    частотный анализ текста
    распределить кандидатов по категориям (культмасс, правозащита, неопределившиеся)

'''

In [ ]:
# векторный анализ текста:
#     посчитать частотность слов в тексте, нормировать по евклидовой норме (СДЕЛАТЬ ЭТО ПО СОЗЫВАМ)
#     убрать союзы, предлоги и тд
'''
документ = программа
коллекция = программы за один созыв

или

документ = программы за один созыв
коллекция = программы за все созывы

классификация текстов регрессией:
    распределить кандидатов по классам (культмасс, правозащита, неопределившиеся)

'''

In [1]:
import pandas as pd
import re
import numpy as np

In [46]:
programs_df = pd.read_csv('../library: main/ssmm_elections_candidates_1_11.csv', sep='\t', encoding='utf-8')
programs_df = programs_df[['conv_x_name','program_txt','convocation_no']][programs_df['program_txt'].notna()].reset_index().drop(['index'],axis=1)
programs_df.head()


,conv_x_name,program_txt,convocation_no
0,1_Левин_Андрей,"Активный участник и организатор различных мероприятий (воспоминания о любимой школе), готов работать, творить, уверен, что все получится.)",1
1,1_Кириллова_Полина,"Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована. Увлекаюсь музыкой, для большего развития записалась в Академический хор. Начала заниматься спортивными бальными танцами. Но в спорте симпатизирую футболу. Планирую посетить еще некоторые клубы Университета и найти, что также может меня затянуть. Хотелось бы внести свой клад в развитие Университета. План работы: 1.Всестороннее развитие студентов. Нас кругом окружает бескультурье и невежество, общество просто гниет. Многие не могут отстаивать свои права, а причина в том, что они просто их не знают. И нужно что-то с этим делать. Поэтому хочется поднять вопрос о всестороннем развитии студентов. Если мы, допустим, обучаемся на Мехмате, это не означает, что мы должны всё своё время отдавать математике и только ей. (В расписании 1 курса в этом учебном году стояли только одни математические дисциплины и физкультура, во втором семестре появились английский язык и БЖД,. Это уже хорошо, но нам убрали историю, это что не такой важный предмет, мы не должны знать истории своей страны?А потом говорят, что нынешнее поколение необразованное.) Почему бы не ввести дополнительный предмет, но при условии, что студент сможет сам для себя выбрать, что ему интересно? Или дополнительные курсы для желающих, возможно, конечно, что такие курсы проводятся, но тогда встает другой вопрос, почему многие студенты не информированы об этом, ведь просто повесить объявление – это мало. Так же можно приглашать государственных деятелей, людей творческих профессий и т. д. Чтобы они делились своим опытом, общались со студентами, отвечали на их вопросы. 2. Мероприятия внутри потоков. Считаю важным поддерживать дружескую атмосферу внутри потоков. Это небольшие ячейки всего университета, поэтому нужно начать с них, нужно делать их крепче и сплоченней. Предлагаю проводить состязательные мероприятия между группами, игры, что ускорит процесс сближения студентов, как в самих группах, так и внутри потоков. 3.Предоставление большей информации о внеучебной деятельности и о клубах МГУ. 4.Общежития. Улучшение санитарных условий. Обустройство общежитий. Нормальный тренажерный зал и душ. Создание клуба в общежитии для организации праздников.",1
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно отправляю вам свой небольшой рассказ, но возможности сделать это ранее просто не представлялось. Итак, есть несколько пунктов, которые необходимо постоянно поддерживать и развивать. Большинство из них известно всем студентам, на них останавливаться я не буду, так как методы решения проблем там идентичны. Общежитие. К сожалению, сам я не очень хорошо знаю всех плюсов и минусов нынешнего общежития для ММ. Но по рассказам однокурсников, проблемы возникают, в первую очередь, с насекомыми. На кухне шастают тараканы, в комнатах клопы, от которых утром просыпаешься весь в укусах, а порой и вообще невозможно уснуть. Эти вопросы несомненно нужно решать общими усилиями СтудСовета и самого общежития. Студенческая жизнь. Очень слабый поток информации доходит до студентов ММ о каких-либо развлекательных и культурных мероприятиях. В первую очередь, необходимо увеличить потом этой информации, а так же дать студентам факультета больше свободы для организации подобных праздников. Пожалуй это основные два пункта, на которых я хотел остановиться. Буду рад вступить в СтудСовет, помогать и развивать Мех-Мат изнутри. Всем удачи. ;)""",1
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я была активным участником жизни класса и гимназии, участвовала в различных городских мероприятиях, играла в школьном театре и была постановщиком танцев. Здесь мне немного этого не хватает..и со мной соглася

In [4]:
len(programs_df)

481

In [37]:
# pd.set_option('display.max_colwidth', None)

In [36]:
# programs_df[programs_df['program_txt'].str.contains('Комментарий студсовета: ')]

In [38]:
# programs_df[['program_txt', 'comment_txt']] = programs_df['program_txt'].str.split('Комментарий студсовета: ', expand=True)
# programs_df


In [39]:
# programs_df[programs_df['comment_txt'].notna()]

## 1. To create 1-token list and to remain only meaningful words

In [51]:
# to create a list of words from all programs

# common list
token_list = []
tokenize_regex = re.compile(r'([\w\-]+)', re.I)

# dictionary of lists (distributed by convocations)
conv_no_list = set(list(programs_df['convocation_no']))
conv_token_list_dict = {}
for conv_no in conv_no_list:
    conv_token_list_dict[conv_no] = []

for i in range(len(programs_df)):
    test_txt = programs_df['program_txt'][i]
    tokens = tokenize_regex.findall(test_txt)
    token_list += [str.lower(t) for t in tokens]
    conv_token_list_dict[programs_df['convocation_no'][i]] += [str.lower(t) for t in tokens]

# немного отредактировать из-за -
token_list = [w for w in token_list if w[0] != '-'] + [w[1:] for w in token_list if w[0] == '-']
for conv_no in conv_no_list:
    conv_token_list_dict[conv_no] = [w for w in conv_token_list_dict[conv_no] if w[0] != '-'] + [w[1:] for w in conv_token_list_dict[conv_no] if w[0] == '-']


# to create a dictionary with word counter

token_dict = {}
for w in token_list:
    token_dict[w] = {'type': '', 'parent': '', 'cnt': 0}

for w in token_list:
    token_dict[w]['cnt'] += 1
        
token_dict = dict(sorted(token_dict.items(), key=lambda item: item[1]['cnt'], reverse=True))
token_dict



{'в': {'type': '', 'parent': '', 'cnt': 1849},
 'и': {'type': '', 'parent': '', 'cnt': 1577},
 'я': {'type': '', 'parent': '', 'cnt': 902},
 'на': {'type': '', 'parent': '', 'cnt': 655},
 'с': {'type': '', 'parent': '', 'cnt': 450},
 'не': {'type': '', 'parent': '', 'cnt': 447},
 'что': {'type': '', 'parent': '', 'cnt': 417},
 'по': {'type': '', 'parent': '', 'cnt': 294},
 'для': {'type': '', 'parent': '', 'cnt': 257},
 'меня': {'type': '', 'parent': '', 'cnt': 244},
 'а': {'type': '', 'parent': '', 'cnt': 228},
 'мехмата': {'type': '', 'parent': '', 'cnt': 226},
 'бы': {'type': '', 'parent': '', 'cnt': 223},
 'как': {'type': '', 'parent': '', 'cnt': 221},
 'это': {'type': '', 'parent': '', 'cnt': 205},
 'за': {'type': '', 'parent': '', 'cnt': 205},
 'хочу': {'type': '', 'parent': '', 'cnt': 203},
 'но': {'type': '', 'parent': '', 'cnt': 186},
 'о': {'type': '', 'parent': '', 'cnt': 183},
 'мне': {'type': '', 'parent': '', 'cnt': 179},
 'студентов': {'type': '', 'parent': '', 'cnt': 17

In [53]:

# by convocations:
    
conv_token_dict = {}
for conv_no in conv_no_list:
    conv_token_dict[conv_no] = {}
    for w in conv_token_list_dict[conv_no]:
        conv_token_dict[conv_no][w] = {'type': '', 'parent': '', 'cnt': 0}

for conv_no in conv_no_list:
    for w in conv_token_list_dict[conv_no]:
        conv_token_dict[conv_no][w]['cnt'] += 1

for conv_no in conv_no_list:
    conv_token_dict[conv_no] = dict(sorted(conv_token_dict[conv_no].items(), key=lambda item: item[1]['cnt'], reverse=True))
conv_token_dict



{1: {'и': {'type': '', 'parent': '', 'cnt': 105},
  'в': {'type': '', 'parent': '', 'cnt': 101},
  'на': {'type': '', 'parent': '', 'cnt': 43},
  'не': {'type': '', 'parent': '', 'cnt': 40},
  'я': {'type': '', 'parent': '', 'cnt': 33},
  'что': {'type': '', 'parent': '', 'cnt': 24},
  'с': {'type': '', 'parent': '', 'cnt': 22},
  'о': {'type': '', 'parent': '', 'cnt': 21},
  'для': {'type': '', 'parent': '', 'cnt': 20},
  'но': {'type': '', 'parent': '', 'cnt': 18},
  'это': {'type': '', 'parent': '', 'cnt': 16},
  'меня': {'type': '', 'parent': '', 'cnt': 14},
  'студентов': {'type': '', 'parent': '', 'cnt': 14},
  'мгу': {'type': '', 'parent': '', 'cnt': 13},
  'по': {'type': '', 'parent': '', 'cnt': 13},
  'бы': {'type': '', 'parent': '', 'cnt': 12},
  'а': {'type': '', 'parent': '', 'cnt': 12},
  'как': {'type': '', 'parent': '', 'cnt': 12},
  'факультета': {'type': '', 'parent': '', 'cnt': 12},
  'за': {'type': '', 'parent': '', 'cnt': 11},
  'все': {'type': '', 'parent': '', 'cn

In [16]:
# предлоги
prepositions_list = [
    # Классификация по происхождению и строению
# Непроизводные (древнейшие предлоги) — 
    'в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через', 'с', 'у', 'за', 'над', 'об', 'обо', 'под', 'про', #'для',
# Производные предлоги образовались в более позднее время от слов других частей речи и подразделяются на:
# наречные — 
    'вблизи', 'вглубь', 'вдоль', 'возле', 'около', 'вокруг', 'впереди', 'после',
# отымённые — 
    'посредством', 'путём', 'путем', 'насчёт', 'насчет', 'ввиду', # в роли, в зависимости от, по поводу, по случаю, в течение и др.;
# глагольные (образованы были в большинстве своём скорее от деепричастий, нежели собственно глаголов) — 
    'благодаря', 'спустя', # несмотря на, 

    # Классификация по структуре
# Простые (состоят из одного слова): 
    'в', 'с', 'со', 'к', 'у', 'над', 'на', 'перед', 'при',
# сложные (состоят из двух простых, соединённых дефисом): 
    'из-под', 'из-за', 'по-над',
# составные (состоят из нескольких слов): 
#     несмотря на, в отличие от, в связи с, 

    # Классификация по употреблению с падежами существительных (валентность)
# с одним падежом: 
    'без', 'до', 'у', 'ради', 'к', 'про', 'через', 'сквозь', 'над', 'перед', 'при', #'для', 
# с двумя падежами: 
    'в', 'на', 'о', 'между', 'за', 'под',
# с тремя падежами: 
    'по', 'с',

    # Классификация по отношениям
# пространственные — 
    'около', 'в', 'среди', 'на',
# временные — 
#     в течение, в продолжение и др.;
# причинные — 
    'ввиду', 'благодаря', 'вследствие', # в связи, из-за и др.;
# целевые — 
   #'для',
# объектные — 
    'о',
# уступки — 
#     несмотря на
    
    
    'вне',
    ''
]

# союзы
conjunctions_list = [
    'и', 'да', 'или', 'либо', 'ли', 'а', 'да', 'но', 'зато', 'однако', 'же', 'тоже', 'также', 
    'притом', 'причём', 'причем', 'именно', 'что', 'чтобы', 'чтоб', 'как', 
    'когда', 'лишь', 'едва', 'где', 'куда', 'откуда', 'столько', 'настолько', 'так', 'такой', # до такой степени, до того,  
    'как', 'словно', 'будто', 'точно', 'если', 'коли', 'ежели', # как будто, как бы, так как, потому что, если бы, 
    'что', 'хотя', 'хоть', 'пускай', 'чтобы', 'дабы', # несмотря на то, с тем чтобы, так что
    'только', 
    ''
#     то есть
#     т.е.
    
#     сочинительные:
# соединительные (и, да (= и), не только … но и, также, тоже, и … и, ни … ни, как … так и, сколько … столько и)
# разделительные (или, или … или, либо, либо … либо, то … то, то ли … то ли, не то … не то)
# противительные (а, да (= но), но, зато, однако, же, однако же, все же)
# градационные (не только… но и, не столько… сколько, не то чтобы… а)
# присоединительные (тоже, также, да и, и, притом, причём)
# пояснительные (то есть, а именно)

    # подчинительные:
# изъяснительные (что, чтобы, как, …)

    # обстоятельственные
# времени (когда, лишь, едва, …)
# места (где, куда, откуда, …)
# образа действия, меры, степени (столько, настолько, так, до такой степени, до того, такой, …)
# сравнения (как, как будто, словно, будто, точно, как бы)
# причины (так как, потому что, …)
# условия (если, если бы, коли, ежели, если … то,…)
# уступки (несмотря на то, что, хотя, хоть, пускай, …)
# цели (чтобы, дабы, с тем чтобы, …)
# следствия (так что)
    
]

# местоимения
pronouns_list = [
#     'я', 'меня', 'мне', 'меня', 'мной', 'мною', 'мой', 'мои', 'своей','свой','себя','себе','свою','свои','своё','свой', 
    'ты', 'тебя', 'тебе', 'тебя', 'тобой', 'тобою', 
#     'вы', 'вашу', 'ваша',
    'он', 'оно', 'его', 'него', 'ему', 'нему', 'им', 'ним', 'нём', 'нем',
    'она', 'её', 'ее', 'неё', 'нее', 'ей', 'ней', 'ею', 'нею',
    'кто', 'что', 'какой', 'каков', 'чей', 'который', 'этот', 'эта', 'это', 'эти', 'тот', 'та', 'то', 'те', 
    'такой', 'такая', 'такое', 'такие', 'таков', 'такова', 'таково', 'таковы', 'сей', 'сия', 'сие', 'сии',
    'все', 'весь', 'всяк', 'всякий', 'любой', 'каждый', 'сам', 'самый', 'другой', 'иной', 
    'никто', 'ничто', 'некого', 'нечего', 'нисколько', 'никакой', 'ничей', 'некто', 'нечто', 'некий', 'некоторый', 'несколько',
    
#     разное
    'нашего', 'том', 'всех', 'всё', 'которые', 'всего', 'этом', 'почему', 'что-то', 'помимо', 
    'них', 'потому', 'эту', 'даже', 'тем', 'этой', 'ещё', 'еще', 'здесь', 'этим', 'всем', 'ведь', 'той', 
    'во-первых', 'во-вторых', 'в-третьих', 'моя', 'всему', 'кое-какие', 'никак', 'наших', 'моё', 'всё-таки', 'наша', 
    'своим', 'кем', 'там', 'чего', 'кого', 'моей', 'ваши', 'вот', 'таких', 'другие', 'как-то', 
    'сама', 'этого', 'нашем', 'теми', 'кому', 'тот', 'сей', 'сей', 'нас', 
    ''
#     неопределённые местоимения с приставкой кое- или постфиксами -то, -либо, -нибудь
]

# наречия
adverbs_list = [
    'только', 'поэтому', 'очень', 'более',
    ''
]

# частицы
particles_list = [
    'бишь', 'бы', 'да', 'де', 'дескать', 'же', 'ишь', 'не', 'небось', 'нет', 'неужели', 'нехай', 'ни', 'таки', 'уж'
#     -ка, -то
]

# другой мусор
garbage_list = [
    '-', '1', '2', '3', '4', '5', '6', '', '', '', '', '', '', ''
]

# full_gabage_list = prepositions_list + conjunctions_list + pronouns_list + adverbs_list + particles_list + garbage_list


In [17]:
def part_of_speech(token_list):
    for w in token_list:
        if w in prepositions_list:
            token_dict[w]['type'] = 'preposition'
        elif w in conjunctions_list:
            token_dict[w]['type'] = 'conjunction'
        elif w in pronouns_list:
            token_dict[w]['type'] = 'pronoun'
        elif w in adverbs_list:
            token_dict[w]['type'] = 'adverb'
        elif w in particles_list:
            token_dict[w]['type'] = 'particle'
        elif w in garbage_list:
            token_dict[w]['type'] = 'garbage'
        else:
            token_dict[w]['type'] = 'meaningful'
    return token_dict

part_of_speech(token_dict)

{'в': {'type': 'preposition', 'parent': '', 'cnt': 1849},
 'и': {'type': 'conjunction', 'parent': '', 'cnt': 1577},
 'я': {'type': 'meaningful', 'parent': '', 'cnt': 902},
 'на': {'type': 'preposition', 'parent': '', 'cnt': 655},
 'с': {'type': 'preposition', 'parent': '', 'cnt': 450},
 'не': {'type': 'particle', 'parent': '', 'cnt': 447},
 'что': {'type': 'conjunction', 'parent': '', 'cnt': 417},
 'по': {'type': 'preposition', 'parent': '', 'cnt': 294},
 'для': {'type': 'meaningful', 'parent': '', 'cnt': 257},
 'меня': {'type': 'meaningful', 'parent': '', 'cnt': 244},
 'а': {'type': 'conjunction', 'parent': '', 'cnt': 228},
 'мехмата': {'type': 'meaningful', 'parent': '', 'cnt': 226},
 'бы': {'type': 'particle', 'parent': '', 'cnt': 223},
 'как': {'type': 'conjunction', 'parent': '', 'cnt': 221},
 'это': {'type': 'pronoun', 'parent': '', 'cnt': 205},
 'за': {'type': 'preposition', 'parent': '', 'cnt': 205},
 'хочу': {'type': 'meaningful', 'parent': '', 'cnt': 203},
 'но': {'type': 'co

In [13]:
# kek = morph.parse('студсовет')[0]
# gent = kek.inflect({'gent'})
# gent.word

In [14]:
# morph.parse('думающему')[0].normal_form

In [15]:
# morph.parse('думающему')[0].inflect({'sing', 'nomn'}).word

In [18]:
# в parent положить инфинитивную форму слов

# pip install pymorphy2
# https://pymorphy2.readthedocs.io/en/latest/user/guide.html#inflection
    
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

meaningful_token_dict = {w: token_dict[w] for w in token_dict if token_dict[w]['type'] == 'meaningful'}
# meaningful_token_dict

for w in meaningful_token_dict:
    meaningful_token_dict[w]['parent'] = morph.parse(w)[0].normal_form
    
meaningful_token_dict

{'я': {'type': 'meaningful', 'parent': 'я', 'cnt': 902},
 'для': {'type': 'meaningful', 'parent': 'для', 'cnt': 257},
 'меня': {'type': 'meaningful', 'parent': 'я', 'cnt': 244},
 'мехмата': {'type': 'meaningful', 'parent': 'мехмат', 'cnt': 226},
 'хочу': {'type': 'meaningful', 'parent': 'хотеть', 'cnt': 203},
 'мне': {'type': 'meaningful', 'parent': 'я', 'cnt': 179},
 'студентов': {'type': 'meaningful', 'parent': 'студент', 'cnt': 173},
 'студсовет': {'type': 'meaningful', 'parent': 'студсовет', 'cnt': 162},
 'уже': {'type': 'meaningful', 'parent': 'уже', 'cnt': 145},
 'факультета': {'type': 'meaningful', 'parent': 'факультет', 'cnt': 141},
 'привет': {'type': 'meaningful', 'parent': 'привет', 'cnt': 137},
 'студсовете': {'type': 'meaningful', 'parent': 'студсовет', 'cnt': 136},
 'мероприятий': {'type': 'meaningful', 'parent': 'мероприятие', 'cnt': 134},
 'студсовета': {'type': 'meaningful', 'parent': 'студсовет', 'cnt': 132},
 'организации': {'type': 'meaningful', 'parent': 'организац

In [10]:
# meaningful_token_dict['стипендиальному']
# meaningful_token_dict['пгас']

In [44]:
meaningful_token__df = pd.DataFrame(meaningful_token_dict).transpose()
meaningful_token__df

,type,parent,cnt
я,meaningful,я,902
для,meaningful,для,257
меня,meaningful,я,244
мехмата,meaningful,мехмат,226
хочу,meaningful,хотеть,203
...,...,...,...
обустройства,meaningful,обустройство,1
продолжение,meaningful,продолжение,1
своевременное,meaningful,своевременный,1
устроить,meaningful,устроить,1


In [45]:
programs_df.head()

,conv_x_name,program_txt,program_symbols_cnt,program_symbols_without_spaces_cnt,program_words_cnt,program_avg_word_length
0,1_Левин_Андрей,"Активный участник и организатор различных мероприятий (воспоминания о любимой школе), готов работать, творить, уверен, что все получится.)",138,122,17,7.176471
1,1_Кириллова_Полина,"Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована. Увлекаюсь музыкой, для большего развития записалась в Академический хор. Начала заниматься спортивными бальными танцами. Но в спорте симпатизирую футболу. Планирую посетить еще некоторые клубы Университета и найти, что также может меня затянуть. Хотелось бы внести свой клад в развитие Университета. План работы: 1.Всестороннее развитие студентов. Нас кругом окружает бескультурье и невежество, общество просто гниет. Многие не могут отстаивать свои права, а причина в том, что они просто их не знают. И нужно что-то с этим делать. Поэтому хочется поднять вопрос о всестороннем развитии студентов. Если мы, допустим, обучаемся на Мехмате, это не означает, что мы должны всё своё время отдавать математике и только ей. (В расписании 1 курса в этом учебном году стояли только одни математические дисциплины и физкультура, во втором семестре появились английский язык и БЖД,. Это уже хорошо, но нам убрали историю, это что не такой важный предмет, мы не должны знать истории своей страны?А потом говорят, что нынешнее поколение необразованное.) Почему бы не ввести дополнительный предмет, но при условии, что студент сможет сам для себя выбрать, что ему интересно? Или дополнительные курсы для желающих, возможно, конечно, что такие курсы проводятся, но тогда встает другой вопрос, почему многие студенты не информированы об этом, ведь просто повесить объявление – это мало. Так же можно приглашать государственных деятелей, людей творческих профессий и т. д. Чтобы они делились своим опытом, общались со студентами, отвечали на их вопросы. 2. Мероприятия внутри потоков. Считаю важным поддерживать дружескую атмосферу внутри потоков. Это небольшие ячейки всего университета, поэтому нужно начать с них, нужно делать их крепче и сплоченней. Предлагаю проводить состязательные мероприятия между группами, игры, что ускорит процесс сближения студентов, как в самих группах, так и внутри потоков. 3.Предоставление большей информации о внеучебной деятельности и о клубах МГУ. 4.Общежития. Улучшение санитарных условий. Обустройство общежитий. Нормальный тренажерный зал и душ. Создание клуба в общежитии для организации праздников.",2306,1981,322,6.152174
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно отправляю вам свой небольшой рассказ, но возможности сделать это ранее просто не представлялось. Итак, есть несколько пунктов, которые необходимо постоянно поддерживать и развивать. Большинство из них известно всем студентам, на них останавливаться я не буду, так как методы решения проблем там идентичны. Общежитие. К сожалению, сам я не очень хорошо знаю всех плюсов и минусов нынешнего общежития для ММ. Но по рассказам однокурсников, проблемы возникают, в первую очередь, с насекомыми. На кухне шастают тараканы, в комнатах клопы, от которых утром просыпаешься весь в укусах, а порой и вообще невозможно уснуть. Эти вопросы несомненно нужно решать общими усилиями СтудСовета и самого общежития. Студенческая жизнь. Очень слабый поток информации доходит до студентов ММ о каких-либо развлекательных и культурных мероприятиях. В первую очередь, необходимо увеличить потом этой информации, а так же дать студентам факультета больше свободы для организации подобных праздников. Пожалуй это основные два пункта, на которых я хотел остановиться. Буду рад вступить в СтудСовет, помогать и развивать Мех-Мат изнутри. Всем удачи. ;)""",1175,1008,168,6.000000
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я была активным участником жизни класса и гимназии, участвовала в

In [50]:
conv_token_dict

{1: ['активный',
  'участник',
  'и',
  'организатор',
  'различных',
  'мероприятий',
  'воспоминания',
  'о',
  'любимой',
  'школе',
  'готов',
  'работать',
  'творить',
  'уверен',
  'что',
  'все',
  'получится',
  'мне',
  'нравится',
  'заниматься',
  'общественной',
  'деятельностью',
  'всячески',
  'пытаюсь',
  'расширять',
  'свой',
  'кругозор',
  'еще',
  'в',
  'школе',
  'организовывала',
  'различные',
  'мероприятия',
  'и',
  'была',
  'сильно',
  'этим',
  'заинтересована',
  'увлекаюсь',
  'музыкой',
  'для',
  'большего',
  'развития',
  'записалась',
  'в',
  'академический',
  'хор',
  'начала',
  'заниматься',
  'спортивными',
  'бальными',
  'танцами',
  'но',
  'в',
  'спорте',
  'симпатизирую',
  'футболу',
  'планирую',
  'посетить',
  'еще',
  'некоторые',
  'клубы',
  'университета',
  'и',
  'найти',
  'что',
  'также',
  'может',
  'меня',
  'затянуть',
  'хотелось',
  'бы',
  'внести',
  'свой',
  'клад',
  'в',
  'развитие',
  'университета',
  'план'

## 2. To calculate weights for 1-TOKENS

In [19]:
conv_x_name_token_dict = {}

for i in range(len(programs_df)):
    tokens = tokenize_regex.findall(programs_df['program_txt'][i])
    conv_x_name_token_dict[programs_df['conv_x_name'][i]] = [str.lower(t) for t in tokens]
    conv_x_name_token_dict[programs_df['conv_x_name'][i]] = [w for w in conv_x_name_token_dict[programs_df['conv_x_name'][i]] if w[0] != '-'] + [w[1:] for w in conv_x_name_token_dict[programs_df['conv_x_name'][i]] if w[0] == '-']
    
# print(conv_x_name_token_dict)


# statistics of infinitive form of 1-tokens - for custom definition of branch 1-token lists

parent_token_dict = {}

for w in meaningful_token_dict:
    parent_token_dict[meaningful_token_dict[w]['parent']] = 0
for w in meaningful_token_dict:
    parent_token_dict[meaningful_token_dict[w]['parent']] += meaningful_token_dict[w]['cnt']
    
parent_token_dict = dict(sorted(parent_token_dict.items(), key=lambda x: x[1], reverse=True))
parent_token_dict



{'я': 1339,
 'студсовет': 460,
 'быть': 384,
 'мехмат': 367,
 'студент': 311,
 'хотеть': 310,
 'год': 300,
 'факультет': 274,
 'для': 257,
 'мероприятие': 232,
 'группа': 231,
 'организация': 229,
 'студенческий': 217,
 'работа': 209,
 'жизнь': 189,
 'свой': 187,
 'вопрос': 178,
 'проблема': 176,
 'заниматься': 171,
 'совет': 169,
 'помогать': 149,
 'уже': 145,
 'день': 144,
 'они': 142,
 'привет': 137,
 'сделать': 136,
 'человек': 133,
 'курс': 131,
 'праздник': 129,
 'мгу': 123,
 'деятельность': 120,
 'время': 119,
 'участвовать': 119,
 'вы': 119,
 'который': 116,
 'мы': 108,
 'проект': 105,
 'есть': 102,
 'участие': 99,
 'мочь': 99,
 'хотеться': 97,
 'себя': 94,
 'работать': 91,
 'новый': 91,
 'идея': 90,
 'хороший': 90,
 'планировать': 80,
 'мой': 79,
 'помочь': 78,
 'тот': 78,
 'активный': 77,
 'образование': 76,
 'наш': 74,
 'университет': 73,
 'являться': 72,
 'много': 71,
 'решение': 70,
 'первый': 70,
 'различный': 69,
 'бытовой': 69,
 'рабочий': 68,
 'интересный': 68,
 'возмо

In [20]:

# добавляем всевозможные вариации по общей родительской форме
def full_list_of_tokens_for_branch(token_dict, branch_parent_tokens):
    full_list_of_tokens = []
    for w in token_dict:
        if token_dict[w]['parent'] in branch_parent_tokens:
            full_list_of_tokens.append(w)
    return full_list_of_tokens


cultmass_tokens = [
    #'мероприятие', 
    'праздник', 'пифагор', 'конкурс', 'посвящение', 'культурно-массовый', 'посвятой', 'культмассовый'
]
sport_tokens = ['спорт', 'чемпионат', 'футбол', 'киберспорт', 'спортивный']
partners_tokens = ['партнёр', 'стажировка']

rights_tokens = [# 'решение', 'решить', 
    'проблема', 'самоуправление', 'право', 'бороться', 'открытость', 'отчётность'
]

education_tokens = ['образование', 'образовательный', 'абитуриент', 'кураторство', 'куратор']
infrastructure_tokens = [
    'бытовой', 'проживание', 'макулатура', 'общежитие', 'стиральный', 'микроволновый', 'курилка', 'таракан', 'клоп', 'поликлиника',
    'туалет', 'чайник', 'общага', 'душевая', 'столовый'
]
inform_tokens = [
    'оповещение', 'оповещать', 'информирование', 'пиар', 'плакат', 'сайт', 'объявление', 'информационный'
]
grants_tokens = ['пгас', 'пгаснуть', 'стипендиальный', 'стипендия']

nothing_tokens = ['насыщенный', 'эффективный', 'лень']

full_cultmass_tokens       = full_list_of_tokens_for_branch(meaningful_token_dict, cultmass_tokens)
full_sport_tokens          = full_list_of_tokens_for_branch(meaningful_token_dict, sport_tokens)
full_partners_tokens       = full_list_of_tokens_for_branch(meaningful_token_dict, partners_tokens)
full_rights_tokens         = full_list_of_tokens_for_branch(meaningful_token_dict, rights_tokens)
full_education_tokens      = full_list_of_tokens_for_branch(meaningful_token_dict, education_tokens)
full_infrastructure_tokens = full_list_of_tokens_for_branch(meaningful_token_dict, infrastructure_tokens)
full_inform_tokens         = full_list_of_tokens_for_branch(meaningful_token_dict, inform_tokens)
full_grants_tokens         = full_list_of_tokens_for_branch(meaningful_token_dict, grants_tokens)
full_nothing_tokens        = full_list_of_tokens_for_branch(meaningful_token_dict, nothing_tokens)

# удалить неоднозначные варианты
# full_rights_tokens.remove('решил')
# full_rights_tokens.remove('решила')

# print('full_cultmass_tokens:', full_cultmass_tokens)
# print('full_sport_tokens:', full_sport_tokens)
# print('full_rights_tokens:', full_rights_tokens)
# print('full_education_tokens:', full_education_tokens)
# print('full_infrastructure_tokens:', full_infrastructure_tokens)
# print('full_inform_tokens:', full_inform_tokens)

In [53]:
# conv_x_name_token_dict['2014_Глунчадзе_Ираклий_Вахтангович']
# programs_df['program_txt'][programs_df.year_no_x_name == '2014_Глунчадзе_Ираклий_Вахтангович']

# kek = programs_df[programs_df.year_no_x_name == '2014_Глунчадзе_Ираклий_Вахтангович']
# kek['program_txt'][23]


In [21]:
# считаем, сколько ключевых слов по каждому направлению встретилось в программе кандидата

def get_flag_by_tokens(conv_x_name_token_dict, year_no_x_name, full_branch_tokens):
    branch_flg = 0
    for t in conv_x_name_token_dict[year_no_x_name]:
        if t in full_branch_tokens:
            branch_flg += 1
    return branch_flg


conv_x_name_weights = {}

for w in conv_x_name_token_dict:    
    conv_x_name_weights[w] = {
        'cultmass_w':       get_flag_by_tokens(conv_x_name_token_dict, w, full_cultmass_tokens), 
        'sport_w':          get_flag_by_tokens(conv_x_name_token_dict, w, full_sport_tokens), 
        'partners_w':       get_flag_by_tokens(conv_x_name_token_dict, w, full_partners_tokens), 
        'rights_w':         get_flag_by_tokens(conv_x_name_token_dict, w, full_rights_tokens), 
        'education_w':      get_flag_by_tokens(conv_x_name_token_dict, w, full_education_tokens), 
        'infrastructure_w': get_flag_by_tokens(conv_x_name_token_dict, w, full_infrastructure_tokens), 
        'inform_w':         get_flag_by_tokens(conv_x_name_token_dict, w, full_inform_tokens),
        'grants_w':         get_flag_by_tokens(conv_x_name_token_dict, w, full_grants_tokens),
        'nothing_w':        get_flag_by_tokens(conv_x_name_token_dict, w, full_nothing_tokens)
    }

# convocation_name_x_flags
conv_x_name_weights_df = pd.DataFrame(conv_x_name_weights).transpose()
conv_x_name_weights_df

,cultmass_w,sport_w,partners_w,rights_w,education_w,infrastructure_w,inform_w,grants_w,nothing_w
1_Левин_Андрей,0,0,0,0,0,0,0,0,0
1_Кириллова_Полина,1,3,0,1,0,3,1,0,0
1_Семенов_Павел,1,0,0,2,0,5,0,0,0
1_Кузнецова_Алёна,0,0,0,0,0,0,0,0,0
1_Кунденко_Вадим,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
11_Терехов_Михаил_Сергеевич,0,0,0,0,0,0,0,0,0
11_Поликарпов_Александр_Валерьевич,0,0,0,0,0,0,0,0,0
11_Каляев_Тимур_Джанбулатович,0,0,0,1,0,0,0,0,0
11_Маммагаджиев_Руслан_Гаджимагомедович,0,0,0,0,0,0,0,0,0


## 3. To calculate weights for  2-TOKENS

In [ ]:
# 2-токены

# сделать поиск 2-токенов и т.д., чтобы выявлять "студсовет МГУ", "университетском уровне" и прочее (на примере Глунчадзе)

In [22]:
# to remove all words except meaningful

conv_x_name_token_dict_meaningful = {}

for name in conv_x_name_token_dict:
    conv_x_name_token_dict_meaningful[name] = [word for word in conv_x_name_token_dict[name] if word in meaningful_token_dict.keys()]
    
# convocation_name_x_token_dict_meaningful['1_Тропин_Александр_Михайлович']


# to replace every word by its parent word

conv_x_name_token_dict_meaningful_parent = {}

for name in conv_x_name_token_dict:
    conv_x_name_token_dict_meaningful_parent[name] = [meaningful_token_dict[word]['parent'] for word in conv_x_name_token_dict_meaningful[name]]
    
# convocation_name_x_token_dict_meaningful_parent['1_Тропин_Александр_Михайлович']


# to create 2-tokens

conv_x_name_2token_dict = {}

for name in conv_x_name_token_dict_meaningful_parent:
    conv_x_name_2token_dict[name] = []
    for i in range(len(conv_x_name_token_dict_meaningful_parent[name]) - 1):
        conv_x_name_2token_dict[name].append([conv_x_name_token_dict_meaningful_parent[name][i], conv_x_name_token_dict_meaningful_parent[name][i+1]])
    
print(conv_x_name_2token_dict)


{'1_Левин_Андрей': [['активный', 'участник'], ['участник', 'организатор'], ['организатор', 'различный'], ['различный', 'мероприятие'], ['мероприятие', 'воспоминание'], ['воспоминание', 'любимый'], ['любимый', 'школа'], ['школа', 'готовый'], ['готовый', 'работать'], ['работать', 'творить'], ['творить', 'уверенный'], ['уверенный', 'получиться']], '1_Кириллова_Полина': [['я', 'нравиться'], ['нравиться', 'заниматься'], ['заниматься', 'общественный'], ['общественный', 'деятельность'], ['деятельность', 'всячески'], ['всячески', 'пытаться'], ['пытаться', 'расширять'], ['расширять', 'свой'], ['свой', 'кругозор'], ['кругозор', 'школа'], ['школа', 'организовывать'], ['организовывать', 'различный'], ['различный', 'мероприятие'], ['мероприятие', 'быть'], ['быть', 'сильно'], ['сильно', 'заинтересовать'], ['заинтересовать', 'увлекаться'], ['увлекаться', 'музыка'], ['музыка', 'для'], ['для', 'больший'], ['больший', 'развитие'], ['развитие', 'записаться'], ['записаться', 'академический'], ['академичес

In [23]:
token2_dict = {}

for name in conv_x_name_2token_dict:
    for t in conv_x_name_2token_dict[name]:
        token2_dict['_'.join(t)] = {'2-token': t, 'cnt': 0}
        
for name in conv_x_name_2token_dict:
    for t in conv_x_name_2token_dict[name]:
        token2_dict['_'.join(t)]['cnt'] += 1

token2_dict = dict(sorted(token2_dict.items(), key=lambda item: item[1]['cnt'], reverse=True))
token2_dict
# print(convocation_2token_dict)

{'студенческий_совет': {'2-token': ['студенческий', 'совет'], 'cnt': 141},
 'я_хотеть': {'2-token': ['я', 'хотеть'], 'cnt': 132},
 'привет_я': {'2-token': ['привет', 'я'], 'cnt': 79},
 'год_я': {'2-token': ['год', 'я'], 'cnt': 74},
 'рабочий_группа': {'2-token': ['рабочий', 'группа'], 'cnt': 63},
 'я_звать': {'2-token': ['я', 'звать'], 'cnt': 61},
 'я_быть': {'2-token': ['я', 'быть'], 'cnt': 57},
 'день_мехмат': {'2-token': ['день', 'мехмат'], 'cnt': 53},
 'повышенный_стипендия': {'2-token': ['повышенный', 'стипендия'], 'cnt': 49},
 'бытовой_вопрос': {'2-token': ['бытовой', 'вопрос'], 'cnt': 49},
 'день_пифагор': {'2-token': ['день', 'пифагор'], 'cnt': 48},
 'студсовет_я': {'2-token': ['студсовет', 'я'], 'cnt': 46},
 'студсовет_мехмат': {'2-token': ['студсовет', 'мехмат'], 'cnt': 45},
 'жизнь_факультет': {'2-token': ['жизнь', 'факультет'], 'cnt': 38},
 'ваш_поддержка': {'2-token': ['ваш', 'поддержка'], 'cnt': 33},
 'организация_праздник': {'2-token': ['организация', 'праздник'], 'cnt':

In [24]:
# 2-tokens
# ПЕРЕБРАЛ до >= 6


full_cultmass_2tokens = [
    'организация_мероприятие', # СПОРНО, но обычно культмасс (добавить веса?)
    'день_мехмат', 'день_пифагор', 'посвящение_первокурсник', 'ночь_мехмат', 'школа_актив', 'конкурс_группа', 'just_dance', 
    'организация_праздник', 'проведение_праздник', 'организация_культурно-массовый', 'организация_различный', 
    'различный_мероприятие', 'всевозможный_мероприятие', 'разнообразный_мероприятие', 'культурно-массовый_мероприятие',
    'группа_праздник', 
    'мероприятие_факультет', 'мехматянский_праздник', 'мехматянский_выезд', 'мехматянский_мероприятие',
    'проведение_день', 'праздник_день', 'мероприятие_посвятой', 'мехмат_посвящение', 
    'настольный_игра', 'досуг_студент'
#     'заниматься_организация', 'помогать_организация', 'участвовать_организация', 'помогать_проведение', 'организация_проведение', 
#     'участие_организация', 
#     'мехмат_день', 'посвятой_день', 'первокурсник_день', 'пифагор_день', 'мероприятие_день', 
]
full_sport_2tokens = [
    'спортивный_мероприятие', 'группа_спорт', 'организация_спортивный'
]
full_partners_2tokens = [
    'работа_партнёр', 'крупный_компания', 'сотрудничество_факультет', 'организовывать_тренинг'
]

full_rights_2tokens = [
    'проблема_студент', 'студенческий_самоуправление', 'решить_проблема', 'отстаивать_интерес', 'решение_проблема', 'право_студент',
    'отстаивать_право', 'свой_поток', 'представлять_интерес', 'представление_интерес', 'мнение_студент', 'представлять_мнение', 
    'отстаивать_мнение', 'прислушиваться_мнение', 'мнение_учитываться', 'инициативный_группа', 'сбор_подпись', 'проблема_университет'
]

full_education_2tokens = [
    'группа_образование', 'качество_образование', 'иностранный_язык', 'учебный_процесс', 'учебный_часть', 'встреча_кафедра',
    'встреча_второкурсник', 'качество_преподавание', 'контроль_качество', 'вопрос_образование', 'опрос_качество', 'курс_иностранный', 
    'для_учёба', 'система_листок'
    #'интерес_учеба'
]


# модернизацией библиотеки мехмата.

# автоматизировать учебную и внеучебную деятельность мехмата. 
# Также развитие культуры на факультете

# Я хочу дать доступ студентам к материалам и информации
# Сделать учёбу приятной)))',


full_infrastructure_2tokens = [
    'бытовой_вопрос', 'группа_бытовой', 'решение_бытовой', 'сбор_макулатура', 'проблема_общежитие', 'улучшение_условие', 
    'бытовой_проблема', 'условие_проживание', 'открытие_переход', 'жилищный_условие', 'обустройство_общежитие', 'обустройство_корпус',
    'оснащение_аудитория', 'утепление_аудитория', 'правило_проход', 'оборудование_место', 'туалетный_бумага', 'высокий_цена', 
    'свободный_вход', 
]
    
full_inform_2tokens = [
    'группа_оповещение', 'повышение_информированность', 'информированность_студент', 'развитие_контент', 'контент_факультет',
    'социальный_сеть', 'обновление_стенд', 'распространять_информация', 'поток_информация'
]

full_grants_2tokens = ['повышенный_стипендия', 'распределение_повышенный', 'распределение_стипендия']

full_unversity_level_2tokens = ['студсовет_мгу', 'совет_мгу', 'самоуправление_мгу']

full_egoistic_2tokens = [
    'ваш_поддержка', 'я_поддержать', 'выбрать_я', 'голосовать_я', 'я_просить', 'свой_кандидатура', 'мой_кандидатура', 'я_обещать', 
    
    'быть_полезный', 'приносить_польза', 'любить_помогать', 'потратить_польза', 'свой_вклад', 'работать_человек', 'помогать_человек',
    'мечтать_поработать', 'помогать_студент',
    
    'мой_интерес', 'искать_себя', 'я_есть', 'попробовать_свой', 'свой_амбиция', 'я_мочь', 'новый_опыт', 'хотеть_стать', 
    'любимый_дело', 'для_я', 'любить_свой', 'я_нужный'
    
]

# ЭГОИСТИЧНАЯ КАТЕГОРИЯ:
# вашу поддержку
# меня поддержите
# выбрать именно меня
# голосуй за меня
# Голосуйте за меня
# быть полезной
# я прошу
# ищу себя
# у меня есть возможность
# Я хочу попробовать свои силы
# приносить пользу в студсовете
# я буду полезен
# я могу
# получить новый опыт
# своей кандидатуре
# я хочу стать частью
# я обещаю
# я хочу продолжить заниматься любимым делом
# работать с людьми
# для меня

# Я считаю себя ответственным, общительным и целеустремлённым
# Политической Партии
# Единая Россия


# также интересные:
# главный_организатор, план_работа, быть_активный, факультет_университет, общежитие_гз, ответственный_секретарь, проект_студсовет
# продолжить_деятельность

# продвигать ваши интересы и пожелания в студсовет
# пожеланиям прислушивались

full_nothing_2tokens = [
    'много_энергия', 
    
    'жизнь_яркий', 'жизнь_приятный', 'сделать_жизнь', 'делать_жизнь', 'общественный_жизнь', 'студенческий_жизнь', 'участие_жизнь',
    'участвовать_жизнь','поучаствовать_жизнь', 'общественный_деятельность', 'общественный_работа',
    
    'новый_идея', 'помочь_продвигать', 'существующий_проект', 'начать_проект', 'новый_инициатива', 'важный_инициатива', 
    'участие_деятельность', 'помогать_развитие',
    
    'работать_благо', 'сделать_полезный', 'изменить_мир', 'мир_хороший', 'изменить_хороший',  
    'ответственный_работа',
    'крутой_команда', 'любить_мехмат', # '', '', ''
]

# НИОЧЕМНАЯ КАТЕГОРИЯ
# она не будет участвовать в рейтинге, но ее нужно будет присваивать тем, у кого в рейтинге пусто

# новые идеи
# продолжить развивать уже начатые проекты
# помочь продвигать уже имеющиеся проекты
# существующими проектами студенческого совета
# завершить начатые проекты и принять участие в координации и реализации новых инициатив
# поддержать важные инициативы
# принять участие в деятельности

# интересно поучаствовать в общественной жизни факультета
# хочу внести свой вклад в улучшение нашего факультета
# работать на благо нашего факультета
# возможность поучаствовать в жизни Университета

# сделать что-то полезное
# создать что-то новое
# сделать её лучше
# изменить мир к лучшему
# изменить ее к лучшему
# 'лучше': {'type': 'meaningful', 'parent': 'хороший', 'cnt': 33},

# ОЛДОВАЯ КАТЕГОРИЯ:
# передавать накопленный опыт


In [25]:
# считаем, сколько ключевых слов по каждому направлению встретилось в программе кандидата

def get_flag_by_2tokens(conv_x_name_2token_dict, year_no_x_name, full_branch_2tokens):
    branch_flg = 0
    for t in conv_x_name_2token_dict[year_no_x_name]:
        if t in [token2_dict[token]['2-token'] for token in full_branch_2tokens]:
            branch_flg += 1
#             break
    return branch_flg


conv_x_name_weights_2 = {}

for w in conv_x_name_2token_dict:    
    conv_x_name_weights_2[w] = {
        'cultmass_w':         get_flag_by_2tokens(conv_x_name_2token_dict, w, full_cultmass_2tokens), 
        'sport_w':            get_flag_by_2tokens(conv_x_name_2token_dict, w, full_sport_2tokens), 
        'partners_w':         get_flag_by_2tokens(conv_x_name_2token_dict, w, full_partners_2tokens), 
        'rights_w':           get_flag_by_2tokens(conv_x_name_2token_dict, w, full_rights_2tokens), 
        'education_w':        get_flag_by_2tokens(conv_x_name_2token_dict, w, full_education_2tokens), 
        'infrastructure_w':   get_flag_by_2tokens(conv_x_name_2token_dict, w, full_infrastructure_2tokens), 
        'inform_w':           get_flag_by_2tokens(conv_x_name_2token_dict, w, full_inform_2tokens), 
        'grants_w':           get_flag_by_2tokens(conv_x_name_2token_dict, w, full_grants_2tokens), 
        'university_level_w': get_flag_by_2tokens(conv_x_name_2token_dict, w, full_unversity_level_2tokens), 
        'egoistic_w':         get_flag_by_2tokens(conv_x_name_2token_dict, w, full_egoistic_2tokens), 
        'nothing_w':          get_flag_by_2tokens(conv_x_name_2token_dict, w, full_nothing_2tokens)
    }

conv_x_name_weights_2

conv_x_name_weights_2_df = pd.DataFrame(conv_x_name_weights_2).transpose()
conv_x_name_weights_2_df



,cultmass_w,sport_w,partners_w,rights_w,education_w,infrastructure_w,inform_w,grants_w,university_level_w,egoistic_w,nothing_w
1_Левин_Андрей,1,0,0,0,0,0,0,0,0,0,0
1_Кириллова_Полина,2,0,0,0,0,1,0,0,0,0,1
1_Семенов_Павел,0,0,0,1,0,0,1,0,0,0,1
1_Кузнецова_Алёна,1,0,0,0,0,0,0,0,0,0,0
1_Кунденко_Вадим,0,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
11_Терехов_Михаил_Сергеевич,0,0,0,0,0,0,0,0,0,0,0
11_Поликарпов_Александр_Валерьевич,0,0,0,0,0,0,0,0,0,0,0
11_Каляев_Тимур_Джанбулатович,0,0,0,0,0,0,0,0,0,0,0
11_Маммагаджиев_Руслан_Гаджимагомедович,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# УБРАТЬ КАКИЕ-ТО ПЕРЕСЕЧЕНИЯ МЕЖДУ 1-токенами и 2-токенами

## 4. To set up flags for 1- and 2-TOKENS

In [26]:
# просуммировать количества из двух таблиц

conv_x_name_common_weights_df = conv_x_name_weights_2_df.add(conv_x_name_weights_df, fill_value=0)
conv_x_name_common_weights_df.head()

,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w
1_Левин_Андрей,1,0,0.0,0,0,0,0,0,0,0,0.0
1_Кириллова_Полина,3,0,0.0,0,1,4,1,0,1,3,0.0
1_Семенов_Павел,1,0,0.0,0,1,5,1,0,3,0,0.0
1_Кузнецова_Алёна,1,0,0.0,0,0,0,0,0,0,0,0.0
1_Кунденко_Вадим,0,0,2.0,0,0,0,2,0,1,0,0.0


In [ ]:
'''
оставить только направления, вес которых не меньше N% от суммарного веса всех направлений кандидата (без nothing)
N = 20% ?
'''

In [27]:
conv_x_name_common_weights_df_2 = conv_x_name_common_weights_df

list_of_branches = [branch for branch in list(conv_x_name_common_weights_df_2) if branch not in ['nothing_w'] ]

conv_x_name_common_weights_df_2['sum_w'] = sum([conv_x_name_common_weights_df_2[branch] for branch in list_of_branches])
conv_x_name_common_weights_df_2.head()



,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w,sum_w
1_Левин_Андрей,1,0,0.0,0,0,0,0,0,0,0,0.0,1.0
1_Кириллова_Полина,3,0,0.0,0,1,4,1,0,1,3,0.0,12.0
1_Семенов_Павел,1,0,0.0,0,1,5,1,0,3,0,0.0,10.0
1_Кузнецова_Алёна,1,0,0.0,0,0,0,0,0,0,0,0.0,1.0
1_Кунденко_Вадим,0,0,2.0,0,0,0,2,0,1,0,0.0,3.0


In [142]:
# list_of_branches

In [143]:
# conv_x_name_common_weights_df_3.head()

In [28]:
def important_flag(x):
    if x is None:
        return 0
    else:
        if x >= 0.15:
            return 1
        else:
            return 0

In [29]:
conv_x_name_common_weights_df_3 = conv_x_name_common_weights_df_2

for branch in list(list_of_branches):
    conv_x_name_common_weights_df_3[branch[:-2]+'_prt'] = conv_x_name_common_weights_df_3[branch]/conv_x_name_common_weights_df_3['sum_w']
    conv_x_name_common_weights_df_3[branch[:-2]+'_flg'] = conv_x_name_common_weights_df_3[branch[:-2]+'_prt'].apply(lambda x: important_flag(x))

conv_x_name_common_weights_df_3 = conv_x_name_common_weights_df_3.drop(list_of_branches+[branch[:-2]+'_prt' for branch in list(list_of_branches)], axis=1)
conv_x_name_common_weights_df_3


,nothing_w,sum_w,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg
1_Левин_Андрей,0,1.0,1,0,0,0,0,0,0,0,0,0
1_Кириллова_Полина,1,12.0,1,0,0,0,0,1,0,0,1,0
1_Семенов_Павел,1,10.0,0,0,0,0,0,1,0,1,0,0
1_Кузнецова_Алёна,0,1.0,1,0,0,0,0,0,0,0,0,0
1_Кунденко_Вадим,2,3.0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11_Терехов_Михаил_Сергеевич,0,0.0,0,0,0,0,0,0,0,0,0,0
11_Поликарпов_Александр_Валерьевич,0,0.0,0,0,0,0,0,0,0,0,0,0
11_Каляев_Тимур_Джанбулатович,0,1.0,0,0,0,0,0,0,0,1,0,0
11_Маммагаджиев_Руслан_Гаджимагомедович,0,0.0,0,0,0,0,0,0,0,0,0,0


In [30]:
conv_x_name_common_weights_df_4 = conv_x_name_common_weights_df_3
conv_x_name_common_weights_df_4['nothing_flg'] = conv_x_name_common_weights_df_4['sum_w'].apply(lambda x: int(1-np.sign(x)))
conv_x_name_common_weights_df_4 = conv_x_name_common_weights_df_4.drop(['nothing_w','sum_w'], axis=1)
conv_x_name_common_weights_df_4


,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
1_Левин_Андрей,1,0,0,0,0,0,0,0,0,0,0
1_Кириллова_Полина,1,0,0,0,0,1,0,0,1,0,0
1_Семенов_Павел,0,0,0,0,0,1,0,1,0,0,0
1_Кузнецова_Алёна,1,0,0,0,0,0,0,0,0,0,0
1_Кунденко_Вадим,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
11_Терехов_Михаил_Сергеевич,0,0,0,0,0,0,0,0,0,0,1
11_Поликарпов_Александр_Валерьевич,0,0,0,0,0,0,0,0,0,0,1
11_Каляев_Тимур_Джанбулатович,0,0,0,0,0,0,0,1,0,0,0
11_Маммагаджиев_Руслан_Гаджимагомедович,0,0,0,0,0,0,0,0,0,0,1


In [31]:
# conv_x_name_common_weights_df_4[conv_x_name_common_weights_df_4['conv_x_name'].apply(lambda x: 'Тропин' in x)]

conv_x_name_common_weights_df_4.filter(like = 'Тропин_', axis=0)

,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
1_Тропин_Александр_Михайлович,0,0,0,0,0,1,0,1,0,0,0
2_Тропин_Александр_Михайлович,0,0,0,0,0,1,0,1,0,1,0
6_Тропин_Александр_Михайлович,0,0,0,0,0,1,0,1,0,1,0


In [32]:
conv_x_name_common_weights_df_4[conv_x_name_common_weights_df_4['university_level_flg'] == 1]


,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
1_Савушкин_Никита_Максимович,0,0,0,0,0,0,0,0,0,1,0
2_Тропин_Александр_Михайлович,0,0,0,0,0,1,0,1,0,1,0
6_Тропин_Александр_Михайлович,0,0,0,0,0,1,0,1,0,1,0
7_Денисов_Илья_Олегович,0,0,0,1,0,0,0,1,0,1,0
8_Денисов_Илья_Олегович,0,0,0,0,0,0,0,1,0,1,0
8_Тонконогов_Александр_Владиславович,1,0,0,0,0,1,0,1,0,1,0
9_Окунев_Фёдор_Алексеевич,0,0,1,0,0,0,0,0,0,1,0
11_Третьяков_Пётр_Максимович,1,0,0,0,0,0,0,0,0,1,0
11_Клюев_Андрей_Игоревич,0,0,0,0,0,0,0,0,0,1,0
11_Дехнич_Кирилл_Андреевич,0,0,0,1,0,0,0,0,0,1,0


In [41]:
programs_df_2 = programs_df

programs_df_2['program_symbols_cnt'] = programs_df_2['program_txt'].apply(lambda x: len(x))
programs_df_2['program_symbols_without_spaces_cnt'] = programs_df_2['program_txt'].apply(lambda x: len(x.replace(' ','')))
programs_df_2['program_words_cnt'] = programs_df_2['program_txt'].apply(lambda x: len(x.split()))
programs_df_2['program_avg_word_length'] = programs_df_2['program_symbols_without_spaces_cnt']/programs_df_2['program_words_cnt']

programs_df_2.head()
# programs_df_2.drop(['program_txt'], axis=1)

,conv_x_name,program_txt,program_symbols_cnt,program_symbols_without_spaces_cnt,program_words_cnt,program_avg_word_length
0,1_Левин_Андрей,"Активный участник и организатор различных мероприятий (воспоминания о любимой школе), готов работать, творить, уверен, что все получится.)",138,122,17,7.176471
1,1_Кириллова_Полина,"Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована. Увлекаюсь музыкой, для большего развития записалась в Академический хор. Начала заниматься спортивными бальными танцами. Но в спорте симпатизирую футболу. Планирую посетить еще некоторые клубы Университета и найти, что также может меня затянуть. Хотелось бы внести свой клад в развитие Университета. План работы: 1.Всестороннее развитие студентов. Нас кругом окружает бескультурье и невежество, общество просто гниет. Многие не могут отстаивать свои права, а причина в том, что они просто их не знают. И нужно что-то с этим делать. Поэтому хочется поднять вопрос о всестороннем развитии студентов. Если мы, допустим, обучаемся на Мехмате, это не означает, что мы должны всё своё время отдавать математике и только ей. (В расписании 1 курса в этом учебном году стояли только одни математические дисциплины и физкультура, во втором семестре появились английский язык и БЖД,. Это уже хорошо, но нам убрали историю, это что не такой важный предмет, мы не должны знать истории своей страны?А потом говорят, что нынешнее поколение необразованное.) Почему бы не ввести дополнительный предмет, но при условии, что студент сможет сам для себя выбрать, что ему интересно? Или дополнительные курсы для желающих, возможно, конечно, что такие курсы проводятся, но тогда встает другой вопрос, почему многие студенты не информированы об этом, ведь просто повесить объявление – это мало. Так же можно приглашать государственных деятелей, людей творческих профессий и т. д. Чтобы они делились своим опытом, общались со студентами, отвечали на их вопросы. 2. Мероприятия внутри потоков. Считаю важным поддерживать дружескую атмосферу внутри потоков. Это небольшие ячейки всего университета, поэтому нужно начать с них, нужно делать их крепче и сплоченней. Предлагаю проводить состязательные мероприятия между группами, игры, что ускорит процесс сближения студентов, как в самих группах, так и внутри потоков. 3.Предоставление большей информации о внеучебной деятельности и о клубах МГУ. 4.Общежития. Улучшение санитарных условий. Обустройство общежитий. Нормальный тренажерный зал и душ. Создание клуба в общежитии для организации праздников.",2306,1981,322,6.152174
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно отправляю вам свой небольшой рассказ, но возможности сделать это ранее просто не представлялось. Итак, есть несколько пунктов, которые необходимо постоянно поддерживать и развивать. Большинство из них известно всем студентам, на них останавливаться я не буду, так как методы решения проблем там идентичны. Общежитие. К сожалению, сам я не очень хорошо знаю всех плюсов и минусов нынешнего общежития для ММ. Но по рассказам однокурсников, проблемы возникают, в первую очередь, с насекомыми. На кухне шастают тараканы, в комнатах клопы, от которых утром просыпаешься весь в укусах, а порой и вообще невозможно уснуть. Эти вопросы несомненно нужно решать общими усилиями СтудСовета и самого общежития. Студенческая жизнь. Очень слабый поток информации доходит до студентов ММ о каких-либо развлекательных и культурных мероприятиях. В первую очередь, необходимо увеличить потом этой информации, а так же дать студентам факультета больше свободы для организации подобных праздников. Пожалуй это основные два пункта, на которых я хотел остановиться. Буду рад вступить в СтудСовет, помогать и развивать Мех-Мат изнутри. Всем удачи. ;)""",1175,1008,168,6.000000
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я была активным участником жизни класса и гимназии, участвовала в

In [42]:
conv_x_name_common_weights_df_final = conv_x_name_common_weights_df_4.reset_index().rename(columns={'index':'conv_x_name'})
conv_x_name_common_weights_df_final = programs_df_2.merge(conv_x_name_common_weights_df_final, left_on='conv_x_name', right_on='conv_x_name')
conv_x_name_common_weights_df_final


,conv_x_name,program_txt,program_symbols_cnt,program_symbols_without_spaces_cnt,program_words_cnt,program_avg_word_length,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
0,1_Левин_Андрей,"Активный участник и организатор различных мероприятий (воспоминания о любимой школе), готов работать, творить, уверен, что все получится.)",138,122,17,7.176471,1,0,0,0,0,0,0,0,0,0,0
1,1_Кириллова_Полина,"Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована. Увлекаюсь музыкой, для большего развития записалась в Академический хор. Начала заниматься спортивными бальными танцами. Но в спорте симпатизирую футболу. Планирую посетить еще некоторые клубы Университета и найти, что также может меня затянуть. Хотелось бы внести свой клад в развитие Университета. План работы: 1.Всестороннее развитие студентов. Нас кругом окружает бескультурье и невежество, общество просто гниет. Многие не могут отстаивать свои права, а причина в том, что они просто их не знают. И нужно что-то с этим делать. Поэтому хочется поднять вопрос о всестороннем развитии студентов. Если мы, допустим, обучаемся на Мехмате, это не означает, что мы должны всё своё время отдавать математике и только ей. (В расписании 1 курса в этом учебном году стояли только одни математические дисциплины и физкультура, во втором семестре появились английский язык и БЖД,. Это уже хорошо, но нам убрали историю, это что не такой важный предмет, мы не должны знать истории своей страны?А потом говорят, что нынешнее поколение необразованное.) Почему бы не ввести дополнительный предмет, но при условии, что студент сможет сам для себя выбрать, что ему интересно? Или дополнительные курсы для желающих, возможно, конечно, что такие курсы проводятся, но тогда встает другой вопрос, почему многие студенты не информированы об этом, ведь просто повесить объявление – это мало. Так же можно приглашать государственных деятелей, людей творческих профессий и т. д. Чтобы они делились своим опытом, общались со студентами, отвечали на их вопросы. 2. Мероприятия внутри потоков. Считаю важным поддерживать дружескую атмосферу внутри потоков. Это небольшие ячейки всего университета, поэтому нужно начать с них, нужно делать их крепче и сплоченней. Предлагаю проводить состязательные мероприятия между группами, игры, что ускорит процесс сближения студентов, как в самих группах, так и внутри потоков. 3.Предоставление большей информации о внеучебной деятельности и о клубах МГУ. 4.Общежития. Улучшение санитарных условий. Обустройство общежитий. Нормальный тренажерный зал и душ. Создание клуба в общежитии для организации праздников.",2306,1981,322,6.152174,1,0,0,0,0,1,0,0,1,0,0
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно отправляю вам свой небольшой рассказ, но возможности сделать это ранее просто не представлялось. Итак, есть несколько пунктов, которые необходимо постоянно поддерживать и развивать. Большинство из них известно всем студентам, на них останавливаться я не буду, так как методы решения проблем там идентичны. Общежитие. К сожалению, сам я не очень хорошо знаю всех плюсов и минусов нынешнего общежития для ММ. Но по рассказам однокурсников, проблемы возникают, в первую очередь, с насекомыми. На кухне шастают тараканы, в комнатах клопы, от которых утром просыпаешься весь в укусах, а порой и вообще невозможно уснуть. Эти вопросы несомненно нужно решать общими усилиями СтудСовета и самого общежития. Студенческая жизнь. Очень слабый поток информации доходит до студентов ММ о каких-либо развлекательных и культурных мероприятиях. В первую очередь, необходимо увеличить потом этой информации, а так же дать студентам факультета больше свободы для организации подобных праздников. Пожалуй это основные два пункта, на которых я хотел остановиться. Буду рад вступить в СтудСовет, помогать и развив

In [43]:
conv_x_name_common_weights_df_final.to_csv('ssmm_programs_info_conv_1_11.csv', sep='\t', encoding='utf-8', index=False)
conv_x_name_common_weights_df_final.to_csv('../library: main/ssmm_programs_info_conv_1_11.csv', sep='\t', encoding='utf-8', index=False)


# ДАЛЬШЕ ЧЕРНОВИКИ

In [197]:
keke = pd.DataFrame(data=conv_x_name_common_weights_df_4.sum())
keke.T

,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
0,185,94,109,46,57,104,18,133,52,11,53


In [3]:
programs_flags_df = pd.read_csv('ssmm_programs_info_conv_1_11.csv', sep='\t', encoding='utf-8')
programs_flags_df.head()

,conv_x_name,program_txt,program_symbols_cnt,program_symbols_without_spaces_cnt,program_words_cnt,program_avg_word_length,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
0,1_Левин_Андрей,Активный участник и организатор различных меро...,138,122,17,7.176471,1,0,0,0,0,0,0,0,0,0,0
1,1_Кириллова_Полина,Мне нравится заниматься общественной деятельно...,2306,1981,322,6.152174,1,0,0,0,0,1,0,0,1,0,0
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно...",1175,1008,168,6.000000,0,0,0,0,0,1,0,1,0,0,0
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я б...",339,295,45,6.555556,1,0,0,0,0,0,0,0,0,0,0
4,1_Кунденко_Вадим,""" Добрый день! Пару слов о себе. Путь к мехм...",971,822,147,5.591837,0,0,1,0,0,0,0,1,0,0,0


In [8]:
branch_list = [branch for branch in list(programs_flags_df) if '_flg' in branch]
branch_list

['cultmass_flg',
 'education_flg',
 'egoistic_flg',
 'grants_flg',
 'inform_flg',
 'infrastructure_flg',
 'partners_flg',
 'rights_flg',
 'sport_flg',
 'university_level_flg',
 'nothing_flg']

In [5]:
candidates_df = pd.read_csv('../library: main/ssmm_elections_candidates_1_11.csv', sep='\t', encoding='utf-8')
year_stream_df = pd.read_csv('../library: main/ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8')
candidates_df = candidates_df.merge(year_stream_df)
candidates_df.head()

,conv_x_name,name,convocation_no,year_stream_no,academic_group_no,program_txt,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm,classic_year_no,year_no,stream_no,course_level,specialization,short_specialization
0,1_Левин_Андрей,Левин Андрей,1,1.1,101.0,Активный участник и организатор различных меро...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,48.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика
1,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика
2,2_Остроухова_Наталья_Владимировна,Остроухова Наталья Владимировна,2,1.1,101.0,Мне всегда была интересна общественная работа ...,https://vk.com/topic-33327655_27333029,2012-11-04 22:31:00,NaN,1,https://vk.com/wall-33327655_725,2012-11-19 16:13:00,2012-11-04 22:31:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика
3,2_Николаева_Наталия_Михайловна,Николаева Наталия Михайловна,2,1.1,102.0,NaN,https://vk.com/topic-33327655_27333029,2012-11-04 22:31:00,NaN,0,NaN,NaN,2012-11-04 22:31:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика
4,2_Габидов_Александр_Сергеевич,Габидов Александр Сергеевич,2,1.1,104.0,"Я хотел бы вступить в Студенческий Совет, чтоб...",https://vk.com/topic-33327655_27333029,2012-11-04 22:31:00,NaN,1,https://vk.com/wall-33327655_725,2012-11-19 16:13:00,2012-11-04 22:31:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика


In [6]:
full_df = programs_flags_df.merge(candidates_df)
full_df

,conv_x_name,program_txt,program_symbols_cnt,program_symbols_without_spaces_cnt,program_words_cnt,program_avg_word_length,cultmass_flg,education_flg,egoistic_flg,grants_flg,...,elected_flg,result_link,result_dttm,appearance_dttm,classic_year_no,year_no,stream_no,course_level,specialization,short_specialization
0,1_Левин_Андрей,Активный участник и организатор различных меро...,138,122,17,7.176471,1,0,0,0,...,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,1,Младшие курсы,Фундаментальная математика,Математика
1,1_Кириллова_Полина,Мне нравится заниматься общественной деятельно...,2306,1981,322,6.152174,1,0,0,0,...,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,2,Младшие курсы,Фундаментальная математика,Математика
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно...",1175,1008,168,6.000000,0,0,0,0,...,0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,2,Младшие курсы,Фундаментальная математика,Математика
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я б...",339,295,45,6.555556,1,0,0,0,...,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,2,Младшие курсы,Фундаментальная математика,Математика
4,1_Кунденко_Вадим,""" Добрый день! Пару слов о себе. Путь к мехм...",971,822,147,5.591837,0,0,1,0,...,0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1,3,Младшие курсы,Фундаментальная механика,Механика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,11_Терехов_Михаил_Сергеевич,За правду!,10,9,2,4.500000,0,0,0,0,...,1,https://vk.com/wall-33327655_10026,2021-10-21 15:44:00,2021-10-16 10:25:00,6,6,1,Старшие курсы,Фундаментальная математика,Математика
477,11_Поликарпов_Александр_Валерьевич,Был одним из основателей бегового клуба MSU RU...,108,91,18,5.055556,0,0,0,0,...,1,https://vk.com/wall-33327655_10031,2021-10-21 22:02:00,2021-10-16 10:50:00,6,6,3,Старшие курсы,Фундаментальная механика,Механика
478,11_Каляев_Тимур_Джанбулатович,"Всем привет! Я только недавно на мехмате, но х...",249,214,36,5.944444,0,0,0,0,...,1,https://vk.com/wall-33327655_10018,2021-10-20 20:21:00,2021-10-16 11:20:00,m1,1,0,Старшие курсы,Магистратура,Магистратура
479,11_Маммагаджиев_Руслан_Гаджимагомедович,Долгое время был пикчером профкома мехмата. Се...,347,301,47,6.404255,0,0,0,0,...,1,https://vk.com/wall-33327655_10018,2021-10-20 20:21:00,2021-10-16 11:20:00,m1,1,0,Старшие курсы,Магистратура,Магистратура


In [14]:

full_df.groupby(['convocation_no']).agg({branch:'sum' for branch in branch_list})

,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
convocation_no,,,,,,,,,,,
1,8,2,4,0,3,7,0,5,2,1,3
2,14,5,3,4,9,8,0,6,5,1,1
3,19,11,5,5,8,9,0,11,7,0,5
4,25,11,8,6,5,12,2,15,6,0,7
5,20,14,5,6,7,13,2,11,7,1,6
6,24,8,13,13,4,18,7,17,6,1,4
7,16,7,9,5,2,4,4,8,3,1,2
8,14,5,11,2,1,6,1,7,5,2,5
9,18,3,9,2,3,7,1,13,5,1,9


In [15]:
full_df.groupby(['classic_year_no']).agg({branch:'sum' for branch in branch_list})

,cultmass_flg,education_flg,egoistic_flg,grants_flg,inform_flg,infrastructure_flg,partners_flg,rights_flg,sport_flg,university_level_flg,nothing_flg
classic_year_no,,,,,,,,,,,
1,43,12,39,1,6,29,1,41,9,1,14
2,46,21,27,7,16,22,4,24,6,0,12
3,38,23,23,10,11,18,9,26,14,1,6
4,34,25,7,12,13,20,3,19,10,2,9
5,18,6,7,14,9,8,1,12,9,4,7
6,3,2,3,2,0,2,0,6,1,2,3
m1,2,4,1,0,0,5,0,4,2,1,1
m2,1,1,2,0,2,0,0,1,1,0,1


In [ ]:
# идея: 
# занумеровать выдвижения каждого кандидата и проследить динамику направлений в его программах (для 3 и более выдвижений)

### ТОП-3 направлений (не нужно?)

In [146]:
# выделить топ-2 направлений по каждому conv_x_name
# что делать при равенстве весов?

# пока только топ-1:
# conv_x_name_common_weights_df_1 = conv_x_name_common_weights_df

# conv_x_name_common_weights_df_1['top_1'] = conv_x_name_common_weights_df_1[list(conv_x_name_common_weights_df)].idxmax(axis=1)
# conv_x_name_common_weights_df_1[:20]



In [77]:
xx = pd.DataFrame(conv_x_name_common_weights_df).T

top_3 = pd.DataFrame(np.zeros((0,3)), columns=['top1','top2','top3'])
rslt = top_3

for conv_x_name in xx.columns:
    df1row = pd.DataFrame(xx.nlargest(3, conv_x_name).index.tolist(), index=['top1','top2','top3']).T
    rslt = pd.concat([rslt, df1row], axis=0)

top_df = rslt.reset_index().drop(['index'], axis=1)
top_df

# xx.nlargest(3, conv_x_name, keep='all') - оставляет всех, занявших не ниже третьего места (КАК НА ВЫБОРАХ НА ПОТОКЕ)

,top1,top2,top3
0,cultmass_w,education_w,egoistic_w
1,infrastructure_w,cultmass_w,sport_w
2,infrastructure_w,rights_w,cultmass_w
3,cultmass_w,education_w,egoistic_w
4,egoistic_w,nothing_w,rights_w
...,...,...,...
476,cultmass_w,education_w,egoistic_w
477,cultmass_w,education_w,egoistic_w
478,rights_w,cultmass_w,education_w
479,cultmass_w,education_w,egoistic_w


In [78]:
# убрать названия, за которыми стоят нули
# названия колонок, в которых нули

def zero_flg(x):
    if x == 0:
        return True
    else:
        return False

column_cnt = len(list(conv_x_name_common_weights_df))

rslt1 = pd.DataFrame(np.zeros((0,column_cnt)))

for conv_x_name in xx.columns:
    df1row1 = pd.DataFrame(xx[xx[conv_x_name].apply(lambda x: zero_flg(x))].index.tolist()).T
    rslt1 = pd.concat([rslt1, df1row1], axis=0)
rslt1 = rslt1.reset_index().drop(['index'], axis=1)
for i in range(column_cnt):
    rslt1 = rslt1.rename(columns={i: 'b'+str(i)})
zero_branches = rslt1

zero_branches


,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10
0,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w,NaN
1,education_w,egoistic_w,grants_w,partners_w,university_level_w,NaN,NaN,NaN,NaN,NaN,NaN
2,education_w,egoistic_w,grants_w,partners_w,sport_w,university_level_w,NaN,NaN,NaN,NaN,NaN
3,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w,NaN
4,cultmass_w,education_w,grants_w,inform_w,infrastructure_w,partners_w,sport_w,university_level_w,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
476,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w
477,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w
478,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,sport_w,university_level_w,NaN
479,cultmass_w,education_w,egoistic_w,grants_w,inform_w,infrastructure_w,nothing_w,partners_w,rights_w,sport_w,university_level_w


In [79]:
# ЗАДЖОЙНИТЬ ПО ИНДЕКСУ
conv_x_name_common_weights_df_2 = conv_x_name_common_weights_df.reset_index().rename(columns={'index':'conv_x_name'})

conv_x_name_common_weights_df_3 = conv_x_name_common_weights_df_2.join(top_df)
# conv_x_name_common_weights_df_3 = pd.merge(conv_x_name_common_weights_df_2, top_df, left_index=True, right_index=True)
# conv_x_name_common_weights_df_3 = pd.concat([conv_x_name_common_weights_df_2, top_df], axis=1)

conv_x_name_common_weights_df_4 = conv_x_name_common_weights_df_3.join(zero_branches)
# conv_x_name_common_weights_df_4

column_list = ['b'+str(i) for i in range(column_cnt)]

df = conv_x_name_common_weights_df_4[['conv_x_name']+column_list+['top1', 'top2', 'top3']]
for top in ['top1', 'top2', 'top3']:
    df[top+'_check'] = np.where(((df[top] != df.b0) & (df[top] != df.b1) & (df[top] != df.b2) & (df[top] != df.b3) & (df[top] != df.b4) & (df[top] != df.b5) & (df[top] != df.b6) & (df[top] != df.b7) & (df[top] != df.b8) & (df[top] != df.b9) & (df[top] != df.b10)), 1, 0)

#     df[top+'_check'] = np.where((np.all(df[top] != df['b'+str(i)] for i in range(column_cnt))), 1, 0)
#     РАБОТАЕТ НЕ ТАК КАК ХОТЕЛОСЬ БЫ !!!

# https://www.pythonfixing.com/2022/01/fixed-npwhere-with-arbitrary-number-of.html

for top in ['top1', 'top2', 'top3']:
    df[top] = np.where((df[top+'_check'] == 0), None, df[top].apply(lambda x: x[:-2])) #.apply(lambda x: x[:-2])
# df = df.drop(['top1_check','top2_check','top3_check'],axis=1)
conv_x_name_top_branches_df = df[['conv_x_name','top1','top2','top3']]

conv_x_name_top_branches_df


/var/folders/zh/mnvg84rj3_7413ycjzpn5y8r0000gp/T/ipykernel_62556/4068508140.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[top+'_check'] = np.where(((df[top] != df.b0) & (df[top] != df.b1) & (df[top] != df.b2) & (df[top] != df.b3) & (df[top] != df.b4) & (df[top] != df.b5) & (df[top] != df.b6) & (df[top] != df.b7) & (df[top] != df.b8) & (df[top] != df.b9) & (df[top] != df.b10)), 1, 0)
/var/folders/zh/mnvg84rj3_7413ycjzpn5y8r0000gp/T/ipykernel_62556/4068508140.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,conv_x_name,top1,top2,top3
0,1_Левин_Андрей,cultmass,None,None
1,1_Кириллова_Полина,infrastructure,cultmass,sport
2,1_Семенов_Павел,infrastructure,rights,cultmass
3,1_Кузнецова_Алёна,cultmass,None,None
4,1_Кунденко_Вадим,egoistic,nothing,rights
...,...,...,...,...
476,11_Терехов_Михаил_Сергеевич,None,None,None
477,11_Поликарпов_Александр_Валерьевич,None,None,None
478,11_Каляев_Тимур_Джанбулатович,rights,None,None
479,11_Маммагаджиев_Руслан_Гаджимагомедович,None,None,None


In [80]:
# ПРОВЕРКА: чьи программы не попали ни в одну категорию

conv_x_name_without_flags = list(df['conv_x_name'][df['top1'].isna()])

list(programs_df['program_txt'][programs_df['conv_x_name'].isin(conv_x_name_without_flags)])



['"Добрый, позитивный, умный, ответственный (рассказ записал под диктовку первокурсника, который живет на моем этаже(я староста этажа)) То есть, люди меня любят)) И работу свою я выполняю ответственно=)"',
 '"год работы в студсовете"',
 'не приложил к заявке рассказ о себе',
 'не прикрепил к заявке рассказ о себе',
 '«Приветики!:)\nЯ очень активный и жизнерадостный человек(когда высыпаюсь) и хотела бы поделится этим со всеми ребятками с мехмата. У меня большой опыт организатора всяких мероприятий, и было бы неплохо, если бы я продолжила эту деятельность в нашем университете.\nLove&Peace»',
 'не прикрепила к заявке рассказ о себе',
 '«Краткость сестра таланта. Инициативная.»',
 '«Считаю себя очень инициативный человеком , и поэтому очень хочется попробовать себя в студенческом совете. Обожаю организовывать мероприятия и агитировать людей, и надеюсь эти качества пригодятся в студсовете;)»',
 '«Наше будущее за котами! Всем котов!»',
 '«Ответственный и коммуникабельный мехматянин готов воп

In [81]:
len(conv_x_name_without_flags)

21

In [87]:
'''

ПРОЧИТАТЬ ПРОГРАММЫ И СРАВНИТЬ С ТОП-3 НАПРАВЛЕНИЙ

'''

pd.set_option('display.max_colwidth', None)

In [96]:
programs_df.merge(conv_x_name_top_branches_df, left_on='conv_x_name', right_on='conv_x_name', how='left')[:60]

# TO FIX 61: 3_Матвеев_Алексей_Викторович

,conv_x_name,program_txt,program_len_symbols,program_len_symbols_2,program_len_words,program_avg_word_len,top1,top2,top3
0,1_Левин_Андрей,"Активный участник и организатор различных мероприятий (воспоминания о любимой школе), готов работать, творить, уверен, что все получится.)",138,122,17,7.176471,cultmass,None,None
1,1_Кириллова_Полина,"Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована. Увлекаюсь музыкой, для большего развития записалась в Академический хор. Начала заниматься спортивными бальными танцами. Но в спорте симпатизирую футболу. Планирую посетить еще некоторые клубы Университета и найти, что также может меня затянуть. Хотелось бы внести свой клад в развитие Университета. План работы: 1.Всестороннее развитие студентов. Нас кругом окружает бескультурье и невежество, общество просто гниет. Многие не могут отстаивать свои права, а причина в том, что они просто их не знают. И нужно что-то с этим делать. Поэтому хочется поднять вопрос о всестороннем развитии студентов. Если мы, допустим, обучаемся на Мехмате, это не означает, что мы должны всё своё время отдавать математике и только ей. (В расписании 1 курса в этом учебном году стояли только одни математические дисциплины и физкультура, во втором семестре появились английский язык и БЖД,. Это уже хорошо, но нам убрали историю, это что не такой важный предмет, мы не должны знать истории своей страны?А потом говорят, что нынешнее поколение необразованное.) Почему бы не ввести дополнительный предмет, но при условии, что студент сможет сам для себя выбрать, что ему интересно? Или дополнительные курсы для желающих, возможно, конечно, что такие курсы проводятся, но тогда встает другой вопрос, почему многие студенты не информированы об этом, ведь просто повесить объявление – это мало. Так же можно приглашать государственных деятелей, людей творческих профессий и т. д. Чтобы они делились своим опытом, общались со студентами, отвечали на их вопросы. 2. Мероприятия внутри потоков. Считаю важным поддерживать дружескую атмосферу внутри потоков. Это небольшие ячейки всего университета, поэтому нужно начать с них, нужно делать их крепче и сплоченней. Предлагаю проводить состязательные мероприятия между группами, игры, что ускорит процесс сближения студентов, как в самих группах, так и внутри потоков. 3.Предоставление большей информации о внеучебной деятельности и о клубах МГУ. 4.Общежития. Улучшение санитарных условий. Обустройство общежитий. Нормальный тренажерный зал и душ. Создание клуба в общежитии для организации праздников.",2306,1981,322,6.152174,infrastructure,cultmass,sport
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно отправляю вам свой небольшой рассказ, но возможности сделать это ранее просто не представлялось. Итак, есть несколько пунктов, которые необходимо постоянно поддерживать и развивать. Большинство из них известно всем студентам, на них останавливаться я не буду, так как методы решения проблем там идентичны. Общежитие. К сожалению, сам я не очень хорошо знаю всех плюсов и минусов нынешнего общежития для ММ. Но по рассказам однокурсников, проблемы возникают, в первую очередь, с насекомыми. На кухне шастают тараканы, в комнатах клопы, от которых утром просыпаешься весь в укусах, а порой и вообще невозможно уснуть. Эти вопросы несомненно нужно решать общими усилиями СтудСовета и самого общежития. Студенческая жизнь. Очень слабый поток информации доходит до студентов ММ о каких-либо развлекательных и культурных мероприятиях. В первую очередь, необходимо увеличить потом этой информации, а так же дать студентам факультета больше свободы для организации подобных праздников. Пожалуй это основные два пункта, на которых я хотел остановиться. Буду рад вступить в СтудСовет, помогать и развивать Мех-Мат изнутри. Всем удачи. ;)""",1175,1008,168,6.000000,infrastructure,rights,cultmass
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь 

## НУЖНА ТАБЛИЦА С ФЛАГАМИ

In [ ]:
# conv_x_name_common_weights_df
# из этой таблицы сделать флаги для топ-3 направлений

'''
СДЕЛАТЬ ТАК: оставить только направления, вес которых не меньше N% от суммарного веса всех направлений кандидата
N = 20% ?
'''

In [89]:
programs_df_2 = programs_df

programs_df_2['program_len_symbols'] = programs_df_2['program_txt'].apply(lambda x: len(x))
programs_df_2['program_len_symbols_2'] = programs_df_2['program_txt'].apply(lambda x: len(x.replace(' ','')))
programs_df_2['program_len_words'] = programs_df_2['program_txt'].apply(lambda x: len(x.split()))
programs_df_2['program_avg_word_len'] = programs_df_2['program_len_symbols_2']/programs_df_2['program_len_words']

programs_df_2.drop(['program_txt'], axis=1)

,conv_x_name,program_len_symbols,program_len_symbols_2,program_len_words,program_avg_word_len
0,1_Левин_Андрей,138,122,17,7.176471
1,1_Кириллова_Полина,2306,1981,322,6.152174
2,1_Семенов_Павел,1175,1008,168,6.000000
3,1_Кузнецова_Алёна,339,295,45,6.555556
4,1_Кунденко_Вадим,971,822,147,5.591837
...,...,...,...,...,...
476,11_Терехов_Михаил_Сергеевич,10,9,2,4.500000
477,11_Поликарпов_Александр_Валерьевич,108,91,18,5.055556
478,11_Каляев_Тимур_Джанбулатович,249,214,36,5.944444
479,11_Маммагаджиев_Руслан_Гаджимагомедович,347,301,47,6.404255


In [90]:
programs_df_2[programs_df_2['conv_x_name'].apply(lambda x: 'Тропин' in x)].sort_values(['program_len_symbols'],
              ascending = [False]).reset_index().drop(['index'], axis=1)[:20]



,conv_x_name,program_txt,program_len_symbols,program_len_symbols_2,program_len_words,program_avg_word_len
0,1_Тропин_Александр_Михайлович,"""о себе: студент 3 курса, работаю в электронной библиотеке мехмата и на МММФ, посещаю собрания Инициативной группы, в связи с чем уже почти год знаком с проблемами общежитий МГУ и студенческого самоуправления. На спешных декабрьских выборах я прошёл в новый студсовет мехмата как кандидат от второго курса, потому что возможность выбраться от третьего учебная часть срезала, а заняться делом хотелось. о планах: Во-первых, хочется закончить Устав студсовета, который мы (пятеро работавших с декабря членов студсовета) начали писать. Ключевыми пунктами устава я считаю открытость и отчётность студсовета (даты собраний известны заранее, прийти на собрание может любой мехматянин, протоколы собраний и отчёты о встречах публикуются в группе) и возможность официального получения мехматских аудиторий для открытых собраний студентов. Впервые в МГУ эти пункты возможно реализовать на мехмате. Также хотелось бы добавить пункт об отсутствии привилегий для членов студсовета: членство не влечёт за собой асперку в ГЗ (как это практикует студком, в результате чего люди занимают пост не ради работы, а ради комнаты), не гарантирует повышенную стипендию и дополнительные пересдачи и т.д. - люди работают в студсовете за идею. Во-вторых, правила прохода в общежитие ГЗ. Только у мехмата учебные и общажные этажи связаны переходом (А13-Б19), который открывается на три часа утром и необоснованно закрыт всё остальное время, в результате чего лифты загружены. Кроме конкретно нашего перехода существуют проблемы правил прохода в общежитие ГЗ в целом. Согласно новому приказу, гости могут оставаться на ночь в асперках, однако возникнет целый ряд уже этим летом: как будет осуществляться проход после введения турникетов? Смогут ли проходить гости студенческих комнат? Заставят ли студентов оставлять студаки на вахте? Будет ли контроль выхода? Разрешат ли не только оставаться ночью, но и проходить тоже? Как будут согласовываться проход через турникет на втором этаже и через переход А13-Б19, где турникетов нет и не планируется? Моя позиция по этим вопросам: отсутствие контроля выхода, неоставление студаков, гости могут приходить и оставаться в любых комнатах, хозяева которых этого хотят, переход А13-Б19 следует открыть с 8:00 до 22:30 (как на первом этаже). Сильная студенческая организация, говорящая от имени почти двух тысяч студентов и аспирантов и выбранная ими, по вопросам, напрямую касающихся их, сможет добиться улучшений в существующее положение дел. Кроме этого, поддерживаю идею создания рабочей группы по праздникам, в которую смогли бы войти все мехматяне, желающие проводить День мехмата на уровне Дней Физика и Химика. На уровне Студсовета МГУ считаю правильным пропорциональность представительства факультетов по схеме 1-2-3 человека, а также идею о начислении стипендий студентам, имевшим пересдачи, но сдавшим сессию на 4 и 5.""",2842,2436,407,5.985258
1,2_Тропин_Александр_Михайлович,"""Вне студсовета: До создания студсоветов участвовал (и продолжаю участвовать) в деятельности Инициативной группы (сбор подписей против ужесточения правил прохода в общежитие ГЗ). Благодаря участию в ИГ выступаю за открытость и гласность студсовета мехмата, а также разбираюсь в устройстве студенческих организаций МГУ. За прошедший год коснулся деятельности всех трёх крупных студ.организаций МГУ: протест против вступления Студсоюза в ОНФ, выборы председателя ОСК и участие в Студсовете МГУ. В студсовете: В студсовете мехмата с декабря прошлого года; с самого начала являюсь секретарём, пишу и публикую развернутые отчёты о собраниях. Координировал и фактически написал устав и положение студсовета мехмата. Участвовал в разработке финальной версии логотипа. Инициировал принятие студсоветом заявления против назначения председателя Студсоюза директором СУНЦ МГУ, а также написал справочник по студорганизациям, рассказывающий об их функционировании и вкла

In [93]:
programs_df_2[programs_df_2['program_len_words'] >= 10].sort_values(['program_avg_word_len'],
              ascending = [False]).reset_index().drop(['index', 'program_txt'], axis=1) #[:60]

,conv_x_name,program_len_symbols,program_len_symbols_2,program_len_words,program_avg_word_len
0,2_Власов_Артем_Андреевич,98,89,10,8.900000
1,2_Дьяков_Павел_Александрович,349,312,37,8.432432
2,2_Акушевич_Андрей_Дмитриевич,349,312,37,8.432432
3,6_Остроухова_Наталья_Владимировна,178,160,19,8.421053
4,4_Мамедов_Рамин_Кямран_Оглы,185,167,20,8.350000
...,...,...,...,...,...
456,9_Корж_Даниил_Сергеевич,297,247,54,4.574074
457,3_Ариас-Альварес_Владимир_Владимирович,93,77,17,4.529412
458,8_Сиухина_Арина_Алексеевна,455,371,85,4.364706
459,7_Филатов_Кирилл_Андреевич,65,53,13,4.076923


In [94]:
programs_df_2.sort_values(['program_len_symbols'],
              ascending = [False]).reset_index().drop(['index', 'program_txt'], axis=1)[:20]

,conv_x_name,program_len_symbols,program_len_symbols_2,program_len_words,program_avg_word_len
0,1_Тропин_Александр_Михайлович,2842,2436,407,5.985258
1,10_Рахматуллаев_Темурбек_Анасбекович,2676,2337,348,6.715517
2,6_Кудрявцев_Сергей_Сергеевич,2392,2076,330,6.290909
3,10_Кухарчук_Иван_Андреевич,2344,2033,328,6.198171
4,1_Кириллова_Полина,2306,1981,322,6.152174
5,1_Удимов_Даниил_Алексеевич,2225,1907,318,5.996855
6,2_Тропин_Александр_Михайлович,2219,1925,295,6.525424
7,1_Сафина_Диана_Ураловна,2167,1884,283,6.657244
8,5_Савин_Павел_Андреевич,2140,1828,323,5.659443
9,11_Третьяков_Пётр_Максимович,2074,1734,345,5.026087


## ДАЛЬШЕ ФЛАГИ

In [ ]:
# ДАЛЕЕ ДЕЛАЕМ ФЛАГИ, НО МБ ЛУЧШЕ ОСТАВИТЬ ВЕСА

In [93]:

for name_flg in list(convocation_name_x_flags_df):
    convocation_name_x_flags_df[name_flg] = convocation_name_x_flags_df[name_flg].apply(lambda x: np.sign(x))

convocation_name_x_flags_df

# convocation_name_x_flags_df2 = convocation_name_x_flags_df
# convocation_name_x_flags_df2


,cultmass_flg,sport_flg,rights_fight_flg,education_flg,infrastructure_flg,inform_flg
1_Левин_Андрей,1,0,0,0,0,0
1_Кириллова_Полина,1,1,1,0,1,0
1_Семенов_Павел,1,0,1,0,1,0
1_Кузнецова_Алёна,1,0,0,0,0,0
1_Кунденко_Вадим,0,0,1,0,0,0
...,...,...,...,...,...,...
11_Терехов_Михаил_Сергеевич,0,0,0,0,0,0
11_Поликарпов_Александр_Валерьевич,0,0,0,0,0,0
11_Каляев_Тимур_Джанбулатович,0,0,1,0,0,0
11_Маммагаджиев_Руслан_Гаджимагомедович,0,0,1,0,0,0


In [41]:

keke = pd.DataFrame(data=convocation_name_x_flags_df2.sum())
keke.T

,cultmass_flg2,sport_flg2,rights_fight_flg2,education_flg2,infrastructure_flg2,inform_flg2
0,219,33,153,64,75,52


In [42]:
convocation_name_x_flags_df3 = convocation_name_x_flags_df2.reset_index().rename(columns={'index':'conv_x_name'})
convocation_name_x_flags_df3


,conv_x_name,cultmass_flg2,sport_flg2,rights_fight_flg2,education_flg2,infrastructure_flg2,inform_flg2
0,1_Левин_Андрей,1,0,0,0,0,0
1,1_Кириллова_Полина,1,1,1,0,0,0
2,1_Семенов_Павел,1,0,1,0,0,0
3,1_Кузнецова_Алёна,1,0,0,0,0,0
4,1_Кунденко_Вадим,0,0,1,0,0,0
...,...,...,...,...,...,...,...
457,11_Терехов_Михаил_Сергеевич,0,0,0,0,0,0
458,11_Поликарпов_Александр_Валерьевич,0,0,0,0,0,0
459,11_Каляев_Тимур_Джанбулатович,0,0,1,0,0,0
460,11_Маммагаджиев_Руслан_Гаджимагомедович,0,0,1,0,0,0


In [43]:
convocation_name_x_flags_df3.to_csv('ssmm_program_classification.csv', sep='\t', encoding='utf-8', index=False)
convocation_name_x_flags_df3.to_csv('library: main/ssmm_program_classification.csv', sep='\t', encoding='utf-8', index=False)


In [44]:
# ИЛИ ТАК

convocation_name_x_flags_df4 = convocation_name_x_flags_df2.reset_index().rename(columns={'index': 'conv_x_name'})
convocation_name_x_flags_df4['convocation_no'] = convocation_name_x_flags_df4['conv_x_name'].apply(lambda x: int(x[0:x.index('_')]))
convocation_name_x_flags_df4.head()

,conv_x_name,cultmass_flg2,sport_flg2,rights_fight_flg2,education_flg2,infrastructure_flg2,inform_flg2,convocation_no
0,1_Левин_Андрей,1,0,0,0,0,0,1
1,1_Кириллова_Полина,1,1,1,0,0,0,1
2,1_Семенов_Павел,1,0,1,0,0,0,1
3,1_Кузнецова_Алёна,1,0,0,0,0,0,1
4,1_Кунденко_Вадим,0,0,1,0,0,0,1


In [45]:
convocation_name_x_flags_df4.groupby(['convocation_no']).sum()

,cultmass_flg2,sport_flg2,rights_fight_flg2,education_flg2,infrastructure_flg2,inform_flg2
convocation_no,,,,,,
1,11,2,6,1,2,0
2,11,0,5,4,4,5
3,17,2,10,3,7,3
4,31,5,15,9,9,6
5,27,9,14,14,12,13
6,29,4,23,7,15,6
7,16,2,12,3,3,3
8,19,3,9,4,7,1
9,23,2,14,1,3,1


## ДРУГОЕ


In [ ]:
'''
ПРОВЕРКА: где программ меньше, чем кандидатов
'''

In [82]:
test_df = pd.read_csv('library: main/ssmm_elections_candidates_1_11.csv', sep='\t', encoding='utf-8')
# test_df = test_df[test_df['program_txt'].notna()]

test_df_2 = test_df.groupby(['convocation_no','year_stream_no']).agg({'conv_x_name':'count','program_txt':'count'})
# test_df_2[:60]
# test_df.head()
ooops_df = test_df_2[test_df_2['conv_x_name'] != test_df_2['program_txt']]
ooops_df

conv_x_name  program_txt
convocation_no year_stream_no                          
1              1.1                       2            1
               1.2                       4            3
               1.3                       5            2
               2.1                       4            0
               2.2                       3            0
               2.3                       4            1
               3.2                       3            1
               3.3                       3            1
               4.1                       4            1
               4.2                       3            2
               4.3                       2            1
               4.4                       1            0
               5.1                       2            1
               5.3                       2            0
2              1.1                       4            3
               2.1                       1            0
               2.2                       3            1
               3.1                       3            1
               3.2                       2            1
               4.2                       3            2
               4.3                       5            4
               5.2                       2            1
               5.3                       1            0
               5.4                       1            0
3              1.1                       2            1
               3.4                       2            1
4              m2                        1            0

In [95]:
ooops_df_2 = test_df[test_df['program_txt'].isna()].merge(ooops_df, left_on=['convocation_no','year_stream_no'], right_on=['convocation_no','year_stream_no'], how='inner')
ooops_df_2

# была проблема: ПЕРЕЗАПИСАЛИСЬ ПО КЛЮЧУ СЛОВАРЯ((((


,conv_x_name_x,name,convocation_no,year_stream_no,academic_group_no,program_txt_x,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm,conv_x_name_y,program_txt_y
0,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,2,1
1,1_Власов_Артем_Андреевич,Власов Артем Андреевич,1,1.2,107.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,36.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,4,3
2,1_Медведева_Яна_Григорьевна,Медведева Яна Григорьевна,1,1.3,121.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,23.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,5,2
3,1_Тишин_Павел_Владимирович,Тишин Павел Владимирович,1,1.3,122.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,51.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,5,2
4,1_Скобенникова_Юлия,Скобенникова Юлия,1,1.3,125.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,17.0,0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,5,2
5,1_Валиев_Зураб_Владимирович,Валиев Зураб Владимирович,1,2.1,201.0,NaN,https://vk.com/topic-33327655_26089854,2012-02-25 23:49:00,23.0,0,https://vk.com/topic-33327655_26089854,2012-02-28 22:34:00,2012-02-25 23:49:00,4,0
6,1_Гильманов_Роман_Павлович,Гильманов Роман Павлович,1,2.1,202.0,NaN,https://vk.com/topic-33327655_26089854,2012-02-25 23:49:00,26.0,1,https://vk.com/topic-33327655_26089854,2012-02-28 22:34:00,2012-02-25 23:49:00,4,0
7,1_Матвеев_Алексей_Викторович,Матвеев Алексей Викторович,1,2.1,205.0,NaN,https://vk.com/topic-33327655_26089854,2012-02-25 23:49:00,54.0,1,https://vk.com/topic-33327655_26089854,2012-02-28 22:34:00,2012-02-25 23:49:00,4,0
8,1_Шариков_Алексей_Анатольевич,Шариков Алексей Анатольевич,1,2.1,205.0,NaN,https://vk.com/topic-33327655_26089854,2012-02-25 23:49:00,58.0,1,https://vk.com/topic-33327655_26089854,2012-02-28 22:34:00,2012-02-25 23:49:00,4,0
9,1_Костицын_Александр_Алексеевич,Костицын Александр Алексеевич,1,2.2,208.0,NaN,https://vk.com/topic-33327655_26089854,2012-02-25 23:49:00,23.0,1,https://vk.com/topic-33327655_26089854,2012-02-28 22:34:00,2012-02-25 23:49:00,3,0


In [ ]:
def n_grams(tokens, n=2):
    return list(zip(*[tokens[i: len(tokens) - n + i + 1] for i in range(n)]))

print(n_grams('язык', n=2))

In [ ]:
from nltk import ngrams